In [19]:
import libsbml
import re
from sympy import symbols, parse_expr

# filename = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/119_JNK-PATHWAY_source.sbml"
filename = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/218_TCA-CYCLE_source.sbml"

In [20]:
# Open the SBML file and get the qualitative_model model
reader = libsbml.SBMLReader()
document = reader.readSBML(filename)

if document.getNumErrors() > 0:  # Check if there are no errors while reading the SBML files.
    errmsg("Error reading SBML file", document.getErrorLog().toString(), kind="WARNING")

model = document.getModel()  # Check whether a model exists.
if model is None:
    errmsg("No model present in SBML file.", kind="WARNING")

qualitative_model = model.getPlugin("qual")  # Get the qualitative_model part of the model.
if qualitative_model is None:  # Check whether a Qual model exists.
    errmsg("The model does not have the Qual plugin", kind="WARNING")

for species in qualitative_model.getListOfQualitativeSpecies():
    print(species.getId())
    print(species.getName())

csa1916
Succinyl-CoA_space_ligase_complex_mitochondrial_space_matrix
csa1897
malate_space_dehydrogenase_space_2_complex
csa1919
fumarate_space_hydratase_complex
csa1898
PDP2:PDPR_complex
csa1902
isocitrate_space_dehydrogenase_complex
csa1899
Succinyl-CoA_space_ligase_complex_mitochondrial_space_matrix
csa1914
mitochondrial_space_pyruvate_space_carrier_complex
csa1949
pyruvate_space_dehydrogenase_space_complex_complex_mitochondrial_space_matrix
csa1901
PDP1:PDPR_complex
csa1921
PDK_space_isoforms_complex
csa1910
alpha-ketoglutarate_space_dehydrogenase_complex
csa1895
pyruvate_space_dehydrogenase_space_complex_complex_mitochondrial_space_matrix
csa1896
isocitrate_space_dehydrogenase_space_3_complex
sa11864
Ca2+_ion
sa12297
GP4G_simple_molecule
sa11889
fumaric_space_acid_simple_molecule
sa11857
phosphate(3-)_simple_molecule
sa11975
NAD(+)_simple_molecule
sa11930
ATP_simple_molecule
sa11896
NADH_simple_molecule
sa11962
GLUD1
sa11966
H+_ion_mitochondrial_space_matrix
sa11863
GDP_simple_mole

In [ ]:
# read the formulas from transitions and convert them to bnet format.
desc = {}
for transition in qualitative_model.getListOfTransitions():  # Scan all the transitions.
    # Get the output variable
    output = transition.getListOfOutputs()
    if len(output) > 1:  # check whether there is a single output
        errmsg("Multiple outputs assigned. List of outputs", output, kind="WARNING")
    else:
        target = output[0].getQualitativeSpecies()

    # Get the formula
    logic_terms = transition.getListOfFunctionTerms()

    if len(logic_terms) == 0:  # Empty transition in SBML file, skip it.
        continue

    if len(logic_terms) > 1:  # check whether there exists a single formula only, error otherwise
        errmsg("Multiple logic terms present. Number of terms", len(logic_terms), kind="WARNING")
    else:  # Get the SBML QUAL formula
        formula = libsbml.formulaToL3String(logic_terms[0].getMath())
        print(formula)

    # Convert the SBML QUAL formula into bnet syntax before parsing it.
    normal_formula = re.sub(r'\|\|', '|', formula)  # convert || to |
    normal_formula = re.sub(r'&&', '&', normal_formula)  # convert && to &
    # convert (<var> == 1) or <var> == 1 to <var>
    normal_formula = re.sub(r'\(?(\w+)\s*==\s*1\)?', r'\1', normal_formula)
    # convert (<var> == 0) or <var> == 0 to ! <var>
    normal_formula = re.sub(r'\(?(\w+)\s*==\s*0\)?', r'!\1', normal_formula)
    print(normal_formula)
    desc[target] = normal_formula

sa11867 == 1
sa11867
((sa11964 == 1) || (csa1921 == 1) || (sa11896 == 1)) && (csa1895 == 1) && (sa11930 == 1) && (sa11945 == 0)
(sa11964 | csa1921 | sa11896) & csa1895 & sa11930 & !sa11945
(((csa1901 == 1) || (sa11864 == 1) || (sa11932 == 1)) && (csa1949 == 1) && (sa11856 == 1)) || (((csa1898 == 1) || (sa11932 == 1)) && (csa1949 == 1) && (sa11856 == 1))
((csa1901 | sa11864 | sa11932) & csa1949 & sa11856) | ((csa1898 | sa11932) & csa1949 & sa11856)
(sa11854 == 1) && (csa1919 == 1)
sa11854 & csa1919
((sa11945 == 1) && (sa11930 == 1) && (sa11946 == 1) && (sa11979 == 1)) || (((csa1901 == 1) || (sa11864 == 1) || (sa11932 == 1)) && (csa1949 == 1) && (sa11856 == 1)) || (((csa1898 == 1) || (sa11932 == 1)) && (csa1949 == 1) && (sa11856 == 1))
(sa11945 & sa11930 & sa11946 & sa11979) | ((csa1901 | sa11864 | sa11932) & csa1949 & sa11856) | ((csa1898 | sa11932) & csa1949 & sa11856)
(sa11939 == 1) && (sa11966 == 1) && (sa11896 == 1) && (csa1897 == 1)
sa11939 & sa11966 & sa11896 & csa1897
(sa11935 ==

In [ ]:

def errmsg(msg: str, arg="", kind: str = "ERROR") -> None:
    print(f"** {kind}:{msg}: {arg}")

def SBML2bnet(filename):
    # Open the SBML file and get the qualitative_model model
    reader = libsbml.SBMLReader()
    document = reader.readSBML(filename)

    if document.getNumErrors() > 0:  # Check if there are no errors while reading the SBML files.
        errmsg("Error reading SBML file", document.getErrorLog().toString(), kind="WARNING")
        return None

    model = document.getModel()  # Check whether a model exists.
    if model is None:
        errmsg("No model present in SBML file.", kind="WARNING")
        return None

    qualitative_model = model.getPlugin("qual")  # Get the qualitative_model part of the model.
    if qualitative_model is None:  # Check whether a Qual model exists.
        errmsg("The model does not have the Qual plugin", kind="WARNING")
        return None

    # STEP: read the formulas from transitions and convert them to bnet format.
    target_equations = {}
    for transition in qualitative_model.getListOfTransitions():  # Scan all the transitions.
        # Get the output variable
        output = transition.getListOfOutputs()
        if len(output) > 1:  # check whether there is a single output
            errmsg("Multiple outputs assigned. List of outputs", output, kind="WARNING")
            return None
        else:
            target = output[0].getQualitativeSpecies()

        # Get the formula
        logic_terms = transition.getListOfFunctionTerms()

        if len(logic_terms) == 0:  # Empty transition in SBML file, skip it.
            continue

        if len(logic_terms) > 1:  # check whether there exists a single formula only, error otherwise
            errmsg("Multiple logic terms present. Number of terms", len(logic_terms), kind="WARNING")
            return None
        else:  # Get the SBML QUAL formula
            formula = libsbml.formulaToL3String(logic_terms[0].getMath())

        # Convert the SBML QUAL formula into bnet syntax before parsing it.
        normal_formula = re.sub(r'\|\|', '|', formula)  # convert || to |
        normal_formula = re.sub(r'&&', '&', normal_formula)  # convert && to &
        normal_formula = re.sub(r'\b(\w+)\s*==\s*1\b', r'\1', normal_formula)  # convert <var> == 1 to <var>
        normal_formula = re.sub(r'\b(\w+)\s*==\s*0\b', r'!\1', normal_formula)  # convert <var> == 0 to ! <var>
        target_equations[target] = normal_formula
    